In [ ]:
%matplotlib inline


# Error handling

We show how errors are handled within ``dman``.


## Introduction
To run the example you will need the following imports:



In [ ]:
from dataclasses import dataclass
import sys
from tempfile import TemporaryDirectory
import traceback
import dman
from dman import tui
from dman import log

# turn off warnings
log.default_config(level=log.CRITICAL)

Since `dman` is designed to store results generated by experiments, it 
always attempts to store as much information as possible. This implies 
that exceptions are handled internally. Often such exceptions 
can occur when encountering unserializable types.



In [ ]:
@dataclass
class Address:
    street: str
    number: int
    zip_code: int
    city: str
    country: str

    def __str__(self):
        return f'{self.street} {self.number}, {self.zip_code} {self.city}, {self.country}'

home = Address('Kasteelpark Arenberg', 10, 3001, 'Leuven', 'Belgium')
employee = {'name': 'John Doe', 'home': home}

dman.save('employee', employee)
tui.walk_directory(dman.mount('employee'), show_content=True)

As you can see only the serializable parts of the data structure were stored. 
The address has been replaced with a specification of the issue encountered.
When we load the employee from disk again we get the following:



In [ ]:
recovered = dman.load('employee')
print(recovered['home'])

Usually a warning is raised when serialization fails:



In [ ]:
with dman.log.logger_context(level=log.WARNING):
    dman.serialize(employee)

The warning will tell you where the serialization failed. In this case 
it specifies `@dict.Address` to indicate that the `Address` object 
was stored in a dictionary. 

<div class="alert alert-info"><h4>Note</h4><p>If you do want an exception thrown you can do get one as follows:

```python
try:
    dman.serialize(employee, context=dman.BaseContext(validate=True))
except dman.ValidationError as e:
    msg = traceback.format_exception(*sys.exc_info())
    print(''.join(msg))
```
      This is only recommended during debugging, since it interrupts 
      serialization while it is in progress and can therefore result 
      in loss of large amounts of data.</p></div>



Usually the log messages should provide enough information to 
fix any issues. We nonetheless continue this example with more 
detailed examples describing the different things that can go wrong during 
serialization and what you can expect as output. 
We also illustrate how data can be recovered even if initial errors 
were present in the implementation.

We will be doing so in terms of the fundamental datastructures involved 
in `dman`. To understand how these fit into the bigger picture 
take a look at the other examples.



## Serializables
Serialization
^^^^^^^^^^^^^^^^^



We first serialize an unserializable type:



In [ ]:
class Base:
    ...


ser = dman.serialize(Base())
tui.print_serialized(ser)

Next let's try serialization of an object with an erroneous implementation
of `__serialize__`.



In [ ]:
@dman.serializable(name="base")
class Base:
    def __serialize__(self):
        raise RuntimeError("Invalid implementation")

    @classmethod
    def __deserialize__(cls, ser):
        ...


ser = dman.serialize(Base())
tui.print_serialized(ser)

After deserialization we can still view the traceback resulting in the exception:



In [ ]:
err = dman.deserialize(ser)
print(err)

We also detect `__serialize__` methods with invalid signatures.



In [ ]:
@dman.serializable(name="base")
class Base:
    # `__serialize__` should either have no or two arguments besides `self`.
    def __serialize__(self, arg1, arg2):
        return 'Base'

    @classmethod
    def __deserialize__(cls, ser):
        raise RuntimeError('Invalid implementation')


ser = dman.serialize(Base())
tui.print_serialized(ser)

### Deserialization
Now we move on to deserialization.



In [ ]:
@dman.serializable(name="base")
class Base:
    def __serialize__(self):
        return '<base>'

    @classmethod
    def __deserialize__(cls, ser):
        raise RuntimeError('Invalid implementation')


ser = dman.serialize(Base())
tui.print_serialized(ser)

Even though the serialized object is valid, we cannot recover the object.



In [ ]:
err = dman.deserialize(ser)
print(err)

Note how the error contains information about what was serialized.
The error is serializable.



In [ ]:
ser = dman.serialize(err)
tui.print_serialized(ser)

Moreover when we try to deserialize it again, now with a valid 
class definition, things work. This sometimes allows for data restoration.



In [ ]:
@dman.serializable(name="base")
class Base:
    def __serialize__(self):
        return '<base>'

    @classmethod
    def __deserialize__(cls, ser):
        return cls()

base = dman.deserialize(ser)
print(base)

## Records

Since a `record` is also serializable it should suppress errors as well
such that the data structure is not damaged by a single erroneous item.
We provide an overview of the errors that typically occur 
and how they are represented within the result of the serialization.

Let's begin with a storable class that cannot write to disk



In [ ]:
@dman.storable(name='base')
class Base:
    def __write__(self, path: str):
        raise RuntimeError('Cannot write to disk.')

    @classmethod
    def __read__(cls, path: str):
        return cls()

rec = dman.record(Base())

If we try to serialize the object without a context we run into some issues



In [ ]:
ser = dman.serialize(rec)
tui.print_serialized(ser)

And when serializing it with a context we run into another.



In [ ]:
root = TemporaryDirectory()
ctx = dman.Context.from_directory(root.name)
ser = dman.serialize(rec, context=ctx)
tui.print_serialized(ser)

When we cannot write instead then we get the following behavior:



In [ ]:
@dman.storable(name='base')
class Base:
    def __write__(self, path: str):
        with open(path, 'w') as f:
            f.write('<base>')

    @classmethod
    def __read__(cls, path: str):
        raise RuntimeError('Cannot read from disk.')

rec = dman.record(Base(), preload=True)
ser = dman.serialize(rec, context=ctx)
tui.print_serialized(ser)

When deserializing we get the following.



In [ ]:
dser = dman.deserialize(ser, context=ctx)
print(dser)
print(dser.content)

Note how the contents are still unloaded. We could fix `Base` and try loading 
again. We also simulate a more advanced scenario where we first serialize 
the invalid record again, reproducing what would happen 
if objects became invalid and the remainder of the data structure 
was saved to disk again. We also define a corrected version of `Base`.



In [ ]:
ser = dman.serialize(dser, context=ctx)
tui.print_serialized(ser)
tui.walk_directory(root.name, show_content=True, console=tui.Console(width=200))

@dman.storable(name='base')
class Base:
    def __write__(self, path: str):
        with open(path, 'w') as f:
            f.write('<base>')

    @classmethod
    def __read__(cls, path: str):
        return cls()

Now we can load the contents of the original invalid record.



In [ ]:
print(dser.content)

And we can load the contents of the one that was serialized when
it was invalid.



In [ ]:
dser = dman.deserialize(ser, context=ctx)
print(dser)
print(dser.content)